In [ ]:
! pip3 install pandas
! pip3 install matplotlib.pyplot
! pip3 install plotly.express
! pip3 install openpyxl
! pip3 install petl

In [ ]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import petl as etl
import glob as glob
import os

In [ ]:
branch_df = pd.read_csv('final_consumer_behaviour_df.csv')

### Track the most purchased and least purchased products & product categories overall, per region and per county

I took certain columns from the branch_df to start the file reduction process

In [ ]:
product_df = branch_df[['region', 'county', 'product', 'quantity','category']]

In order to make this df useable on Dash, I scaled it right down to only use the files I need.

This file was reduced from 10.9 GB to 563 KB

In [ ]:
regional_product = product_df. groupby(['region', 'county', 'product','category'])['quantity'].sum().sort_values(ascending=False).reset_index()

In [ ]:
regional_product.to_csv('testing_output/product_df.csv')

Below is the process of making sure I output the right data to then use in the dash app

In [ ]:
regional_product_search = regional_product.loc[regional_product['county'] == 'East Lindsey']
top_product = regional_product_search.groupby('category')['quantity'].max(5).reset_index()
bottom_product = regional_product_search.groupby('category')['quantity'].max(5).reset_index()


In [ ]:
top_product.plot.bar(x=('category'), y=["quantity"])

### Track the best performing branches overall per region and per county (performance is measured in both item quantity sold and monetary value of sales made)

In [ ]:
performance_df = branch_df[['region', 'county', 'branch_name', 'quantity','amount_in_gbp']]


This file was reduced from 10.9 GB to 5KB

In [ ]:
regional_performance = performance_df.groupby(['region', 'county', 'branch_name'])[['quantity', 'amount_in_gbp']].sum().sort_values(by='region', ascending=False).reset_index()

regional_performance.to_csv('testing_output/performance_df.csv')

Below is the process of making sure I output the right data to then use in the dash app

In [ ]:
regional_performance['best_performing'] = regional_performance.quantity + regional_performance.amount_in_gbp
regional_performance.sort_values(by='best_performing', ascending=False)
regional_performance['region'].value_counts().sort_values(ascending=False)

In [ ]:
top_regional_performance = regional_performance.loc[regional_performance['region'] == 'London'].nlargest(5,'best_performing')
top_regional_performance.plot.bar(x=('branch_name') , y=['best_performing'])

### Per hour sales for the top 10 branches of 2021 identified

In [ ]:
per_hour_df = branch_df[['branch_name', 'year', 'hour', 'amount_in_gbp']]


This file was reduced from 10.9 GB to 846 KB

In [ ]:
per_hour = per_hour_df.groupby(['year', 'branch_name', 'hour'])['amount_in_gbp'].sum().sort_values(ascending=False).reset_index()

In [ ]:
per_hour.to_csv('testing_output/per_hour_df.csv')

Below is the process of making sure I output the right data to then use in the dash app

In [ ]:
year_filtered_df = per_hour.loc[per_hour['year']== 2020]

In [ ]:
year_filtered_df.sort_values(by=['branch_name', 'hour'])

In [ ]:
branch_hour_grouped_df = year_filtered_df.groupby(['branch_name', 'hour'])['amount_in_gbp'].sum().reset_index()

In [ ]:
top_branches = branch_hour_grouped_df.groupby('branch_name')['amount_in_gbp'].sum().reset_index()
top_branches = top_branches.rename(columns={"amount_in_gbp":"total_gbp"})

In [ ]:
merged_per_hour = branch_hour_grouped_df.merge(top_branches.set_index('branch_name'), on='branch_name').reset_index()

top_merged_per_hour = merged_per_hour.sort_values(['total_gbp', 'hour'], ascending=True).tail(240)
bottom_merged_per_hour = merged_per_hour.sort_values(['total_gbp', 'hour'], ascending=True).head(240)

In [ ]:
top_merged_per_hour = top_merged_per_hour.pivot(index='hour', columns='branch_name', values='amount_in_gbp')
top_merged_per_hour.plot(figsize=(20,10))

In [ ]:
bottom_merged_per_hour = bottom_merged_per_hour.pivot(index='hour', columns='branch_name', values='amount_in_gbp')
bottom_merged_per_hour.plot(figsize=(20,10))

### Identify the top 10 and bottom 10 profitable branches and indicate how profitable they are.

In [ ]:
profit_df = branch_df[['branch_name', 'amount_in_gbp','year']]

This file was reduced from 10.9 GB to 32 KB

In [ ]:
profitable_branches_df = profit_df.groupby(['year', 'branch_name'])['amount_in_gbp'].sum().reset_index()

In [ ]:
profitable_branches_df.to_csv('testing_output/profitable_branches_df.csv')

Below is the process of making sure I output the right data to then use in the dash app

In [ ]:
year_filtered_df = profitable_branches_df.loc[profitable_branches_df['year']== 2020]

In [ ]:
year_filtered_df.sort_values(by=['year','amount_in_gbp'],ascending=False)

In [ ]:
top_year_filtered_df = year_filtered_df.head(10)
bottom_year_filtered_df = year_filtered_df.tail(10)
top_year_filtered_df

I needed to import the branch_expenses, take the columns I needed and create a total expenses column, the join to existing df

In [ ]:
branch_expenses_df = pd.read_csv('branch_expenses.csv')

In [ ]:
branch_expenses_df
branch_expenses_df['total_expenses'] = branch_expenses_df.apply(lambda row: row.operational_cost + row.staff_bonuses + row.misc_expenses + row.waste_cost, axis=1)

In [ ]:
total_branch_expenses =branch_expenses_df.groupby('branch_name')['total_expenses'].sum().reset_index()
branch_total_df = top_year_filtered_df.merge(total_branch_expenses.set_index('branch_name'), on='branch_name') 

In [ ]:
branch_total_df['profitability'] = branch_total_df.apply(lambda row: row.amount_in_gbp - row.total_expenses, axis=1)
branch_total_df['profitability'] = branch_total_df.amount_in_gbp - branch_total_df.total_expenses
branch_total_df.sort_values(by='profitability', ascending=False)

In [ ]:
branch_total_df.sort_values(by='profitability', ascending=False)
top_profitability_stores = branch_total_df.head(10)
bottom_profitability_stores = branch_total_df.tail(10)
top_profitability_stores


In [ ]:
top_profitability_stores.plot.bar(x=('branch_name'), y=['total_expenses', 'amount_in_gbp'], rot=0, subplots=True)
plt.xlabel(None)
plt.ylabel(None)
plt.title('Top Profitable Stores 2021')
plt.xticks(rotation=90)

In [ ]:
bottom_profitability_stores.plot.bar(x=('branch_name'), y=['profitability'])
plt.ylim(bottom= 1.00000e+09)
plt.xlabel('Branch Name')
plt.ylabel('Profitability')
plt.title('Least Profitable Stores 2021')